In [ ]:
# 데이터 병합 merged_df로
car_df['brand'] = car_df['title'].apply(lambda x: x.split()[0])

brand_df.rename({'title':'brand'},axis=1,inplace=True)

brand_df['brand'] = brand_df['brand'].apply(lambda x: x.upper())

merged_df = car_df.merge(brand_df, on='brand' ,how='inner')

In [ ]:
# Engine 컬럼전처리
merged_df['Engine'] = merged_df['Engine'].apply(lambda x: float(str(x).replace('L','')))

In [ ]:
# Emission Class 컬럼 전처리
merged_df['Emission Class'] = merged_df[~(merged_df['Emission Class'].isna())]['Emission Class'].apply(lambda x: x.split()[-1])

merged_df['Emission Class'] = merged_df[~(merged_df['Emission Class'].isna())]['Emission Class'].apply(lambda x: int(x))

In [ ]:
# Service history 변수의 결측치는 'Unknown'으로 채우기
merged_df['Service history'].fillna('Unknown', inplace=True) 

In [ ]:
# 결측치 비율 0.382696이 무조건 삭제할 정도로 높은지 모르겠다.
# 이 피처가 중요한 피처라면 삭제하면 안되지 않나.
# price와의 상관관계를 보니 중요한 거 같다. 대체를 해보자.
# Registration_Year가 높으면 (최신일수록),  Previous Owners는 적을 거 같다.
# 뭔가 관계로 대체하고싶지만 복잡하니까 모드값으로

## 빈도 그래프
correlations = merged_df.dropna(subset=['Previous Owners']).corr()['Previous Owners']
correlations.drop('Previous Owners', errors='ignore') 

In [ ]:
sns.countplot(data=merged_df, x='Previous Owners')
mode_value = merged_df['Previous Owners'].mode()[0]
merged_df['Previous Owners'].fillna(mode_value, inplace=True) 

In [ ]:
# 결측치 변수가 일정 개수 이상 포함된 데이터는 제거합니다.
# 행 기준으로 결측치 4개인 데이터 제거
# 3개는?..
null_4_index = merged_df[merged_df['Engine'].isna()&merged_df['Doors'].isna()&merged_df['Seats'].isna()&merged_df['Seats'].isna()].index
merged_df = merged_df.drop(index=null_4_index)
merged_df = merged_df.dropna()

In [ ]:
# 크게 보이는 이상치는 Mileage(miles)의 1개
sns.scatterplot(data=merged_df, x=merged_df.index, y=merged_df['Mileage(miles)'])

In [ ]:
# 제거
merged_df = merged_df.drop(index=merged_df[merged_df['Mileage(miles)'] > 1e6].index)

In [ ]:
# 원핫인코딩
# 일단 object 조사 1.title, 2.Fuel type, 3.Body type, 4.Gearbox, 5.Service history, 6.brand, 7.country
# 이 중에서 unique값이 많으면 줄인다.
merged_df.info()


In [ ]:
# title은 가격예측할 때 되게 중요할 수 있다고 생각이 드는데, 
# 원핫 인코딩이 하고 싶기때문에 brand로 묶인다 생각하고 제거.
merged_df['title'].value_counts()

In [ ]:
# 2.Fuel type #하이브리드를 묶자. 몇개 안됨.
merged_df['Fuel type'].value_counts()

In [ ]:
# 처리
merged_df['Fuel type'] = merged_df['Fuel type'].apply(lambda x: 'Hybrid' if x.split()[-1] == 'Hybrid' else x)

In [ ]:
# 3. Body type # 밴과 픽업트럭.. 
Body_type_ratios = merged_df['Body type'].value_counts()/len(merged_df['Body type'])
merged_df['Body type'] = merged_df['Body type'].apply(lambda x: 'Others' if Body_type_ratios[x] < 0.02 else x)
merged_df['Body type'].value_counts()

In [ ]:
# 4.Gearbox
merged_df['Gearbox'].value_counts()

In [ ]:
# 5.Service history
merged_df['Service history'].value_counts()

In [ ]:
# 6.brand, title까지 지울건데 brand까지 지우면 안되지 않을까요? 너무 많으니까 2프로 이하
brand_ratios = merged_df['brand'].value_counts() / len(merged_df)
merged_df['brand'] = merged_df['brand'].apply(lambda x: 'Others' if brand_ratios[x] < 0.02 else x)

In [ ]:
# 7.country 
country_ratios = merged_df['country'].value_counts()/len(merged_df['country'])
merged_df['country'] = merged_df['country'].apply(lambda x: 'Others' if country_ratios[x] < 0.02 else x)

In [ ]:
# title drop
merged_df_1 = merged_df.drop(columns='title')

In [ ]:
# 원핫인코딩
merged_df_2 = pd.get_dummies(merged_df_1, columns=['Fuel type', 'Body type', 'Gearbox', 'Service history', 'brand', 'country'], drop_first= True)

In [ ]:
# 스케일링
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(merged_df_2)
ss_df = pd.DataFrame(ss.transform(merged_df_2), columns = merged_df_2.columns)

In [ ]:
# PCA
ss_df.head()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(2)

pca_result = pca.fit_transform(merged_df_2)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
merged_df.groupby('country')['brand'].nunique()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 상관관계 계산
correlation_matrix = merged_df.corr()

# 상관관계 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 상관관계 계산
correlation_matrix = merged_df_1.corr()

# 상관관계 히트맵 시각화
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)